In [26]:
from __future__ import print_function,division,absolute_import
import tables
from keras.utils import to_categorical
from keras.models import model_from_json
from custom_layers import Conv1D_linearphase, DCT1D
from heartnet_v1 import reshape_folds
import os

In [21]:
checkpoint_name = "/media/taufiq/Data1/heart_sound/models/fold1+compare 2018-05-05 17:04:36.995687/weights.0007-0.8148.hdf5"
foldname = 'fold1+compare'
fold_dir = '/media/taufiq/Data1/heart_sound/feature/segmented_noFIR/'

## Load Model

In [18]:
model_dir = checkpoint_name[:checkpoint_name.find('fold')]
log_name = checkpoint_name[checkpoint_name.find('fold'):checkpoint_name.find('weights')-1]
print(log_name)

fold1+compare 2018-05-05 17:04:36.995687


In [19]:
if os.path.isdir(model_dir+log_name):
    print("Model directory found")
    if os.path.isfile(os.path.join(model_dir+log_name,"model.json")):
        print("model.json found. Importing")
    else:
        raise ImportError("model.json not found")

Model directory found
model.json found. Importing


In [20]:
with open(model_dir+log_name+"/model.json") as json_file:
    loaded_model_json = json_file.read()
model = model_from_json(loaded_model_json,{'Conv1D_linearphase':Conv1D_linearphase,'DCT1D':DCT1D})
model.load_weights(checkpoint_name)

## Load Data

In [27]:
############## Importing data ############
feat = tables.open_file(fold_dir + foldname + '.mat')
x_train = feat.root.trainX[:]
y_train = feat.root.trainY[0, :]
x_val = feat.root.valX[:]
y_val = feat.root.valY[0, :]
train_parts = feat.root.train_parts[:]
val_parts = feat.root.val_parts[0, :]

############## Relabeling ################

for i in range(0, y_train.shape[0]):
    if y_train[i] == -1:
        y_train[i] = 0  ## Label 0 for normal 1 for abnormal
for i in range(0, y_val.shape[0]):
    if y_val[i] == -1:
        y_val[i] = 0

############# Parse Database names ########

train_files = []
for each in feat.root.train_files[:][0]:
    train_files.append(chr(each))
print(len(train_files))
val_files = []
for each in feat.root.val_files[:][0]:
    val_files.append(chr(each))
print(len(val_files))

################### Reshaping ############

x_train, y_train, x_val, y_val = reshape_folds(x_train, x_val, y_train, y_val)
y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)

3283
515
(93942, 2500, 1)
(93942, 1)
(15511, 2500, 1)
(15511, 1)


In [44]:
for dataset in ['a','b','c','d','e','f','x']:
    mask = np.asarray(val_files) == dataset
    temp = np.asarray(val_parts)
    print(np.sum(temp[mask]))

3389.0
312.0
403.0
186.0
4657.0
692.0
5872.0
